In [170]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

# 기본 데이터 세팅

In [171]:
블록원데이터 = pd.DataFrame({"블록명":["S1", "S2", "S3", "S4"], 
                    "중량": [50, 60, 30, 20], 
                    "가로": [10, 10, 5, 5], 
                    "세로":[10, 12, 6, 6], 
                    "표준공기":[2, 3, 1, 2], 
                    "납기":["2024-02-15", "2024-02-15", "2024-02-15", "2024-02-15"],
                    "정반배치":[0, 0, 0, 0]})
블록원데이터.head()

,블록명,중량,가로,세로,표준공기,납기,정반배치
0,S1,50,10,10,2,2024-02-15,0
1,S2,60,10,12,3,2024-02-15,0
2,S3,30,5,6,1,2024-02-15,0
3,S4,20,5,6,2,2024-02-15,0


In [172]:
정반원데이터 = pd.DataFrame({"정반명":["A", "B", "C"],
                   "가능중량":[100, 100, 30],
                   "가로":[10, 10, 5],
                   "세로":[20, 10, 10],
                   })
정반원데이터.head()

,정반명,가능중량,가로,세로
0,A,100,10,20
1,B,100,10,10
2,C,30,5,10


In [173]:
start_date = datetime(2024, 2, 1)
end_date = datetime(2024, 2, 28)
날짜집합  = pd.date_range(start=start_date, end=end_date, freq='D')
날짜집합

DatetimeIndex(['2024-02-01', '2024-02-02', '2024-02-03', '2024-02-04',
               '2024-02-05', '2024-02-06', '2024-02-07', '2024-02-08',
               '2024-02-09', '2024-02-10', '2024-02-11', '2024-02-12',
               '2024-02-13', '2024-02-14', '2024-02-15', '2024-02-16',
               '2024-02-17', '2024-02-18', '2024-02-19', '2024-02-20',
               '2024-02-21', '2024-02-22', '2024-02-23', '2024-02-24',
               '2024-02-25', '2024-02-26', '2024-02-27', '2024-02-28'],
              dtype='datetime64[ns]', freq='D')

In [174]:
블록집합 = 블록원데이터["블록명"].unique().tolist()
블록집합

['S1', 'S2', 'S3', 'S4']

In [175]:
최초정반집합 = 정반원데이터["정반명"].unique().tolist()
최초정반집합

['A', 'B', 'C']

In [176]:
쌩전체경우의수 = len(날짜집합) * len(블록집합) * len(최초정반집합)
쌩전체경우의수

336

# 블록 데이터 전처리 함수

In [177]:
# df apply 함수에 복수개의 변수를 던져주기
import functools
def unpack_df_columns(func):
    @functools.wraps(func)
    def _unpack_df_columns(*args, **kwargs):
        series = args[0]
        return func(*series.values)
    return _unpack_df_columns

In [178]:
@unpack_df_columns
def 최소요구착수일구하기(납기, 공기):
    result = pd.to_datetime(납기) - timedelta(days=int(공기))
    return result.date()

In [179]:
착수일가중치, 공기가중치, 크기가중치 = 0.7, 0.5, 0.5

@unpack_df_columns
def 블록우선순위구하기(날순, 공순, 크순):
    global 착수일가중치, 공기가중치, 크기가중치
    result = np.round((날순*착수일가중치 + 공순*공기가중치 + 크순*크기가중치)/3,1)
    return result


In [180]:
def 블록데이터전처리(블록원데이터):
    df1 = 블록원데이터.copy()
    df1["납기"] = pd.to_datetime(df1["납기"])
    df1["사이즈"] = df1.eval("가로*세로")
    df1["최소요구착수일"] = df1[["납기", "표준공기"]].apply(최소요구착수일구하기, axis=1)
    df1["날짜순서"] = df1["최소요구착수일"].rank()
    df1["공기순서"] = df1["표준공기"].rank(ascending=False)
    df1["크기순서"] = df1["사이즈"].rank(ascending=False)
    df1["우선순위"] = df1[["날짜순서", "공기순서", "크기순서"]].apply(블록우선순위구하기, axis=1)
#     df1["길이리스트"] = df1[["가로", "세로"]].apply(길이리스트만들기, axis=1)
#     df1["최장길이"] = df1["길이리스트"].apply(lambda x : max(x))
#     df1["최소길이"] = df1["길이리스트"].apply(lambda x : min(x))
#     df1["면적리스트"] = df1["길이리스트"].apply(면적리스트구하기)
    df1 = df1.sort_values(by=["우선순위"])
    return df1

In [181]:
df1 = 블록데이터전처리(블록원데이터)
df1

,블록명,중량,가로,세로,표준공기,납기,정반배치,사이즈,최소요구착수일,날짜순서,공기순서,크기순서,우선순위
1,S2,60,10,12,3,2024-02-15,0,120,2024-02-12,1.0,1.0,1.0,0.6
0,S1,50,10,10,2,2024-02-15,0,100,2024-02-13,2.5,2.5,2.0,1.3
3,S4,20,5,6,2,2024-02-15,0,30,2024-02-13,2.5,2.5,3.5,1.6
2,S3,30,5,6,1,2024-02-15,0,30,2024-02-14,4.0,4.0,3.5,2.2


# 정반 데이터 전처리 함수

In [182]:
중량가중치, 크기가중치 = 0.5, 0.7

@unpack_df_columns
def 정반우선순위구하기(중순, 크순):
    global 중량가중치, 크기가중치
    result = np.round((중순*중량가중치 + 크순*크기가중치)/3,1)
    return result

@unpack_df_columns
def 길이리스트만들기(가로, 세로):
    result = []
    result.append(가로)
    result.append(가로)
    result.append(세로)
    result.append(세로)
    return result

def 면적리스트구하기(길이리스트):
    면적리스트 = []
    if 길이리스트[0] == 길이리스트[1]:
        면적리스트.append(길이리스트[0]*길이리스트[3])
    else:
        면적리스트  = []
        면적리스트.append(길이리스트[0]*길이리스트[3])
        면적리스트.append(길이리스트[1]*길이리스트[2])
    return 면적리스트

In [183]:
def 정반데이터전처리(정반원데이터):
    df2 = 정반원데이터.copy()
    df2["사이즈"] = df2.eval("가로 * 세로")
    df2["중량순서"] = df2["가능중량"].rank(ascending=False)
    df2["크기순서"] = df2["사이즈"].rank(ascending=False)
    df2["우선순위"] = df2[["중량순서", "크기순서"]].apply(정반우선순위구하기, axis=1)
    df2["길이리스트"] = df2[["가로", "세로"]].apply(길이리스트만들기, axis=1)
    df2["최장길이"] = df2["길이리스트"].apply(lambda x : max(x))
    df2["최소길이"] = df2["길이리스트"].apply(lambda x : min(x))
    df2["면적리스트"] = df2["길이리스트"].apply(면적리스트구하기)
    df2 = df2.sort_values(by=["우선순위"])
    return df2

In [184]:
df2 = 정반데이터전처리(정반원데이터)
df2

,정반명,가능중량,가로,세로,사이즈,중량순서,크기순서,우선순위,길이리스트,최장길이,최소길이,면적리스트
0,A,100,10,20,200,1.5,1.0,0.5,"[10, 10, 20, 20]",20,10,[200]
1,B,100,10,10,100,1.5,2.0,0.7,"[10, 10, 10, 10]",10,10,[100]
2,C,30,5,10,50,3.0,3.0,1.2,"[5, 5, 10, 10]",10,5,[50]


# 배치달력 함수

In [185]:
def create_init_calendar(날짜집합, 정반집합):
    배치달력 = pd.DataFrame()
    배치달력.index = 날짜집합
   
    for 정반 in 정반집합:
        배치달력[f"{정반}"] = 0

    return 배치달력

In [186]:
def update_배치달력(배치달력, 정반명, 착수날짜, 필요공기, 정반집합):
    
    신규칼럼리스트 = 정반집합.copy()
    try:
        for 현칼럼 in 배치달력.columns:
            신규칼럼리스트.remove(현칼럼)

        for 신규칼럼 in 신규칼럼리스트:
            배치달력[f"{신규칼럼}"] = 0

        시점인덱스 = list(배치달력.index.strftime('%Y-%m-%d')).index(착수날짜)
        배치달력[f"{정반명}"].iloc[시점인덱스:시점인덱스+필요공기] = 1
        return 배치달력
    except:
        return 배치달력

# 공기달력 함수

In [187]:
def create_공기달력(배치달력, 날짜집합, 정반집합):
    total_list = []

    for 정반 in 정반집합:
        검토대상 = 배치달력[f"{정반}"].tolist()

        new_list = []
        new_num = 0
        for idx, i in enumerate(검토대상):
            if i == 0:
                new_num = new_num  + 1
                new_list.append(new_num)
            else:
                new_list.append(0)
                new_num = 0
        total_list.append(new_list)
        
    new_total = []
    for original_list in total_list:

        result_list = []
        group = []
        for num in original_list:
            if num == 0 and group:
                result_list.extend(reversed(group))
                group = []
            group.append(num)

        result_list.extend(reversed(group))

        new_total.append(result_list)

    공기달력 = pd.DataFrame()
    공기달력.index = 날짜집합

    for idx, 정반 in enumerate(정반집합):
        공기달력[f"{정반}"] =  new_total[idx]

    for 정반 in 정반집합:
        if 공기달력[f"{정반}"][0]== 0:
            공기달력[f"{정반}"] = 공기달력[f"{정반}"].shift(1)
        else:
            pass
    공기달력.fillna(0, inplace=True)
    return 공기달력

# 공백순서달력(first_zero) 함수

In [188]:
def create_공백순서달력(배치_달력, 정반집합, 날짜집합):
    total = []

    for 정반 in 배치_달력.columns.tolist():
        
        input_list = 배치_달력[f"{정반}"].tolist()
        
        counter = 1
        result_list = []

        for idx, x in enumerate(input_list):

            if idx == 0:
                if x == 1:
                    result_list.append(0)
                else:
                    result_list.append(counter)
                    counter += 1

            else:   
                if input_list[idx-1] == 1 and x == 0:
                    result_list.append(counter)
                    counter += 1
                else:
                    result_list.append(0)

        total.append(result_list)

    공백순서달력 = pd.DataFrame()
    공백순서달력.index = 날짜집합

    for idx, 정반 in enumerate(배치_달력.columns.tolist()):
        공백순서달력[f"{정반}"] =  total[idx]

    return 공백순서달력

# 착수가능일찾기 함수

In [189]:
def 착수가능일찾기(공기달력, 공백순서달력, 정반, 표준공기):
        
    first_zeros = []
    
    for idx, i in enumerate(공백순서달력[f"{정반}"].tolist()):
        if i != 0:
            first_zeros.append(공백순서달력.index.strftime('%Y-%m-%d').values[idx])

    for idx, 착수가능일 in enumerate(first_zeros):
        
        
        착수가능일인덱스 = list(공기달력.index.strftime('%Y-%m-%d')).index(first_zeros[idx])
        착수가능일의확보가능공기 = 공기달력[f"{정반}"].iloc[착수가능일인덱스]
        
        if 착수가능일의확보가능공기 > 표준공기:
            
            return 착수가능일
        
        else:
            pass

# 달력생성 및 업데이트 연습

## 최초달력생성

In [190]:
배치달력 = create_init_calendar(날짜집합, 최초정반집합)
배치달력.iloc[:2,:] = 1   ## 날짜 계산의 편의를 위해 초반 2일을 배치상태로 설정
배치달력

,A,B,C
2024-02-01,1,1,1
2024-02-02,1,1,1
2024-02-03,0,0,0
2024-02-04,0,0,0
2024-02-05,0,0,0
2024-02-06,0,0,0
2024-02-07,0,0,0
2024-02-08,0,0,0
2024-02-09,0,0,0
2024-02-10,0,0,0


In [191]:
정반집합 = 배치달력. columns.tolist()
공기달력 = create_공기달력(배치달력, 날짜집합, 정반집합)
공기달력

,A,B,C
2024-02-01,0.0,0.0,0.0
2024-02-02,0.0,0.0,0.0
2024-02-03,26.0,26.0,26.0
2024-02-04,25.0,25.0,25.0
2024-02-05,24.0,24.0,24.0
2024-02-06,23.0,23.0,23.0
2024-02-07,22.0,22.0,22.0
2024-02-08,21.0,21.0,21.0
2024-02-09,20.0,20.0,20.0
2024-02-10,19.0,19.0,19.0


In [192]:
공백순서달력 = create_공백순서달력(배치달력, 정반집합, 날짜집합)
공백순서달력

,A,B,C
2024-02-01,0,0,0
2024-02-02,0,0,0
2024-02-03,1,1,1
2024-02-04,0,0,0
2024-02-05,0,0,0
2024-02-06,0,0,0
2024-02-07,0,0,0
2024-02-08,0,0,0
2024-02-09,0,0,0
2024-02-10,0,0,0


## 블록배치

In [193]:
착수가능일 = 착수가능일찾기(공기달력, 공백순서달력, "A", 13)
착수가능일

'2024-02-03'

## 달력 업데이트

In [194]:
새정반집합 = ["A","B","C","A_추가"]
배치달력 = update_배치달력(배치달력, "A", "2024-02-10", 2, 새정반집합)
배치달력

,A,B,C,A_추가
2024-02-01,1,1,1,0
2024-02-02,1,1,1,0
2024-02-03,0,0,0,0
2024-02-04,0,0,0,0
2024-02-05,0,0,0,0
2024-02-06,0,0,0,0
2024-02-07,0,0,0,0
2024-02-08,0,0,0,0
2024-02-09,0,0,0,0
2024-02-10,1,0,0,0


In [195]:
공기달력 = create_공기달력(배치달력, 날짜집합, 새정반집합)
공기달력

,A,B,C,A_추가
2024-02-01,0.0,0.0,0.0,28
2024-02-02,0.0,0.0,0.0,27
2024-02-03,7.0,26.0,26.0,26
2024-02-04,6.0,25.0,25.0,25
2024-02-05,5.0,24.0,24.0,24
2024-02-06,4.0,23.0,23.0,23
2024-02-07,3.0,22.0,22.0,22
2024-02-08,2.0,21.0,21.0,21
2024-02-09,1.0,20.0,20.0,20
2024-02-10,0.0,19.0,19.0,19


In [196]:
공백순서달력 = create_공백순서달력(배치달력, 새정반집합, 날짜집합)
공백순서달력

,A,B,C,A_추가
2024-02-01,0,0,0,1
2024-02-02,0,0,0,0
2024-02-03,1,1,1,0
2024-02-04,0,0,0,0
2024-02-05,0,0,0,0
2024-02-06,0,0,0,0
2024-02-07,0,0,0,0
2024-02-08,0,0,0,0
2024-02-09,0,0,0,0
2024-02-10,0,0,0,0


# 배치시나리오
 - 블록접수
 - 정반 매칭, 배치달력 체크 (우선순위가 고려되는 단계 - 매칭정반, 공기확보 가능한 first_zero)
 - 정반 및 블록 사이즈 체크 - 정반쪼개기
 - 정반새리스트업 - 정반별 배치달력, 공기달력생성
 - 기준시점 생산량 계산 (최적화 목적함수)

## 원데이터 불러오기

In [197]:
블록원데이터 = pd.read_excel("./data/data1.xlsx", sheet_name="블록데이터")
블록원데이터.head()

,블록명,중량,가로,세로,표준공기,납기,정반배치
0,S1,50,10,10,2,2024-02-15,0
1,S2,60,10,12,3,2024-02-15,0
2,S3,30,5,6,1,2024-02-15,0
3,S4,20,5,6,2,2024-02-15,0


In [198]:
정반원데이터 = pd.read_excel("./data/data1.xlsx", sheet_name="정반데이터")
정반원데이터.head()

,정반명,가능중량,가로,세로
0,A,100,10,20
1,B,100,10,15
2,C,30,5,10


In [199]:
블록데이터 = 블록데이터전처리(블록원데이터)
블록데이터

,블록명,중량,가로,세로,표준공기,납기,정반배치,사이즈,최소요구착수일,날짜순서,공기순서,크기순서,우선순위
1,S2,60,10,12,3,2024-02-15,0,120,2024-02-12,1.0,1.0,1.0,0.6
0,S1,50,10,10,2,2024-02-15,0,100,2024-02-13,2.5,2.5,2.0,1.5
3,S4,20,5,6,2,2024-02-15,0,30,2024-02-13,2.5,2.5,3.5,1.8
2,S3,30,5,6,1,2024-02-15,0,30,2024-02-14,4.0,4.0,3.5,2.4


In [200]:
정반데이터 = 정반데이터전처리(정반원데이터)
정반데이터

,정반명,가능중량,가로,세로,사이즈,중량순서,크기순서,우선순위,길이리스트,최장길이,최소길이,면적리스트
0,A,100,10,20,200,1.5,1.0,0.5,"[10, 10, 20, 20]",20,10,[200]
1,B,100,10,15,150,1.5,2.0,0.7,"[10, 10, 15, 15]",15,10,[150]
2,C,30,5,10,50,3.0,3.0,1.2,"[5, 5, 10, 10]",10,5,[50]


In [230]:
def 생산계획수립(블록데이터, 정반데이터, 계획시작일, 계획종료일):

    # 결과모음리스트
    배정된블록 = []
    배정된정반 = []
    착수일 = []
    표준공기 = []
    종료일 = []
    조립중량  = []
    
    자식정반고유번호 = 0
    
    수정블록리스트 = 블록데이터["블록명"].tolist()
    수정정반리스트 = 정반데이터["정반명"].tolist()
    
    계획시작일 = datetime(int(계획시작일[:4]), int(계획시작일[5:7]), int(계획시작일[-2:]))
    계획종료일 = datetime(int(계획종료일[:4]), int(계획종료일[5:7]), int(계획종료일[-2:]))
    날짜집합  = pd.date_range(start=계획시작일, end=계획종료일, freq='D')
    
    배치달력 = create_init_calendar(날짜집합, 수정정반리스트)
    공기달력 = create_공기달력(배치달력, 날짜집합, 수정정반리스트)
    공백순서달력 = create_공백순서달력(배치달력, 수정정반리스트, 날짜집합)
    
    for _ in tqdm(range(len(수정블록리스트))):
        
        if 수정블록리스트:
            target_block = 수정블록리스트[0]
        else:
            print("수정블록리스트에 검토대상 잔여블록이 없습니다.")
            break
            
        blk_index = 블록데이터[블록데이터["블록명"]==target_block].index.values[0]
        target_weight = 블록데이터[블록데이터["블록명"]==target_block]["중량"].values[0]
        target_size = 블록데이터[블록데이터["블록명"]==target_block]["사이즈"].values[0]
        least_start_date = 블록데이터[블록데이터["블록명"]==target_block]["최소요구착수일"].values[0]
        target_표준공기 = 블록데이터[블록데이터["블록명"]==target_block]["표준공기"].values[0]
        print(f"타겟블록정보: idx-{blk_index}, name-{target_block}, weight-{target_weight}, size-{target_size}, 최소착수요구일-{least_start_date}, 표준공기-{target_표준공기}")
    
        
        가능정반_dict = {}
        for 정반 in 수정정반리스트:
#             print(f"검토정반: {정반}")
            
            공백순서1인덱스 = list(공백순서달력[f"{정반}"]).index(1)
            공백순서1인덱스의날짜 = 배치달력.index[공백순서1인덱스]
            공백순서1인덱스의날짜의확보가능공기 = 공기달력[f"{정반}"][공백순서1인덱스]
            공백순서1인덱스날짜의공백순서 = 공백순서달력[f"{정반}"][공백순서1인덱스]
            weight_capa = 정반데이터[정반데이터["정반명"]==정반]["가능중량"].values[0]
            size_capa = 정반데이터[정반데이터["정반명"]==정반]["사이즈"].values[0]
            
            try:
                공백순서2인덱스 = list(공백순서달력[f"{정반}"]).index(2)
                print(f"공백순서2인덱스: {공백순서2인덱스}")
                공백순서2인덱스의날짜 = 배치달력.index[공백순서2인덱스]
                공백순서2인덱스의날짜의확보가능공기 = 공기달력[f"{정반}"][공백순서2인덱스]
                공백순서2인덱스날짜의공백순서 = 공백순서달력[f"{정반}"][공백순서2인덱스]
                
                if 공백순서1인덱스의날짜 <= least_start_date and 공백순서1인덱스의날짜의확보가능공기 >= target_표준공기 and weight_capa >= target_weight and size_capa >= target_size and 공백순서1인덱스날짜의공백순서 == 1:
                    가능정반_dict[정반] = 공백순서1인덱스의날짜
                elif 공백순서2인덱스의날짜 <= least_start_date and 공백순서2인덱스의날짜의확보가능공기 >= target_표준공기 and weight_capa >= target_weight and size_capa >= target_size and 공백순서2인덱스날짜의공백순서 == 2:
                    가능정반_dict[정반] = 공백순서2인덱스의날짜
                else:
                    # st.markdown(f"타겟블록{target_block}는 정반 {정반}가 1, 2순위에 해당하지 않음")
                    pass
                
                
                
            except:
                if 공백순서1인덱스의날짜 <= least_start_date and 공백순서1인덱스의날짜의확보가능공기 >= target_표준공기 and weight_capa >= target_weight and size_capa >= target_size and 공백순서1인덱스날짜의공백순서 == 1:
                    가능정반_dict[정반] = 공백순서1인덱스의날짜
                else:
                    # st.markdown(f"타겟블록{target_block}는 정반 {정반}가 1, 2순위에 해당하지 않음")
                    pass
                pass
    
    
        if 가능정반_dict != {}:
            print(f"가능정반 : {가능정반_dict}")
            최선조기착수가능정반 = [key for key, value in 가능정반_dict.items() if value == min(가능정반_dict.values())]   # 여러개일수 있으므로 리스트로 반환
            print(f"최선조기착수 가능정반 : {min(가능정반_dict.values())} ---> {최선조기착수가능정반}")  
            랜덤최선정반 = random.choice(최선조기착수가능정반)


            weight_capa = 정반데이터[정반데이터["정반명"]==랜덤최선정반]["가능중량"].values[0]
            size_capa = 정반데이터[정반데이터["정반명"]==랜덤최선정반]["사이즈"].values[0]

            잔여면적비율 = (size_capa - target_size) / size_capa
            print(f"랜덤선택최선정반 : {랜덤최선정반} - size_capa: {size_capa}, block_size:{target_size}, 잔여면적비율 :{잔여면적비율}")

            if 잔여면적비율 >= 정반쪼개는면적_Thresh:
                새정반이름 = 랜덤최선정반+f"_{자식정반고유번호}"
                자식정반고유번호 += 1
                새정반면적 = size_capa * 잔여면적비율
                기존정반새면적 = size_capa - 새정반면적
                가능정반인덱스 = 정반데이터[정반데이터["정반명"]==랜덤최선정반].index.values[0]

                정반데이터.loc[가능정반인덱스,"사이즈"] = 기존정반새면적
                정반데이터.loc[len(정반데이터)] = {"정반명":새정반이름, "가능중량": weight_capa, "사이즈":새정반면적}

                정반데이터["중량순서"] = 정반데이터["가능중량"].rank(ascending=False)
                정반데이터["크기순서"] = 정반데이터["사이즈"].rank(ascending=False)
                정반데이터["우선순위"] = 정반데이터[["중량순서", "크기순서"]].apply(정반우선순위구하기, axis=1)
                정반데이터 = 정반데이터.sort_values(by=["우선순위"])

                print(f"잔여면적비율 {np.round(잔여면적비율,1)*100}%로 30% 이상이므로 정반 쪼개기 - 자식정반이름 :blue[**{새정반이름}**] / 자식정반면적 {새정반면적} / 엄마정반면적-{기존정반새면적}") 
                수정정반리스트.append(새정반이름)  
            else:
                print(f"1순위 정반의 잔여 면적이 Thresh({정반쪼개는면적_Thresh}) 비율보다 작아 쪼갤 수 없습니다.")

            착수가능일 = 착수가능일찾기(공기달력, 공백순서달력, 랜덤최선정반, target_표준공기)
            블록데이터.loc[blk_index, "정반배치"] = 1

            배정결과 = {"블록명": target_block, "정반명": 랜덤최선정반, "착수일": 착수가능일}   
            print(f"최종배정결과 - {배정결과}") 

            배정된블록.append(target_block)
            배정된정반.append(랜덤최선정반)
            착수일.append(착수가능일)
            표준공기.append(target_표준공기)
            original_date = datetime.strptime(착수가능일, "%Y-%m-%d")
            종료날짜 = original_date + timedelta(days=int(target_표준공기)) 
            종료날짜 = 종료날짜.strftime("%Y-%m-%d")
            종료일.append(종료날짜)
            조립중량.append(target_weight)

            print(f"(블록배치전) 수정블록리스트 : {수정블록리스트}, 수정정반리스트 : {수정정반리스트}")
            수정블록리스트.remove(target_block) 
            print(f"(블록배치후) 수정블록리스트 : {수정블록리스트}, 수정정반리스트 : {수정정반리스트}")


            배치달력 =  update_배치달력(배치달력, 랜덤최선정반, 착수가능일, target_표준공기, 수정정반리스트) 
    #         배치달력.iloc[:5,:] = 1    ## 갱신시 하드코딩 2일차까지 강제로 1 채우기
            공기달력 = create_공기달력(배치달력, 날짜집합, 수정정반리스트)
            공백순서달력 = create_공백순서달력(배치달력, 수정정반리스트, 날짜집합)
            
            print("="*50)


    최종배정결과 = pd.DataFrame({"블록명":배정된블록, "정반명":배정된정반, "착수일":착수일, "표준공기":표준공기, "종료일": 종료일, "조립중량": 조립중량})

    return 최종배정결과, 배치달력, 공기달력, 공백순서달력

In [231]:
블록원데이터 = pd.read_excel("./data/data1.xlsx", sheet_name="블록데이터")
정반원데이터 = pd.read_excel("./data/data1.xlsx", sheet_name="정반데이터")
블록데이터 = 블록데이터전처리(블록원데이터)
정반데이터 = 정반데이터전처리(정반원데이터)

result = 생산계획수립(블록데이터, 정반데이터, "2024-02-01", "2024-02-28")
result[0]

100%|████████████████████████████████████████████| 4/4 [00:00<00:00, 50.00it/s]

타겟블록정보: idx-1, name-S2, weight-60, size-120, 최소착수요구일-2024-02-12, 표준공기-3
가능정반 : {'A': Timestamp('2024-02-01 00:00:00', freq='D'), 'B': Timestamp('2024-02-01 00:00:00', freq='D')}
최선조기착수 가능정반 : 2024-02-01 00:00:00 ---> ['A', 'B']
랜덤선택최선정반 : B - size_capa: 150, block_size:120, 잔여면적비율 :0.2
1순위 정반의 잔여 면적이 Thresh(0.3) 비율보다 작아 쪼갤 수 없습니다.
최종배정결과 - {'블록명': 'S2', '정반명': 'B', '착수일': '2024-02-01'}
(블록배치전) 수정블록리스트 : ['S2', 'S1', 'S4', 'S3'], 수정정반리스트 : ['A', 'B', 'C']
(블록배치후) 수정블록리스트 : ['S1', 'S4', 'S3'], 수정정반리스트 : ['A', 'B', 'C']
타겟블록정보: idx-0, name-S1, weight-50, size-100, 최소착수요구일-2024-02-13, 표준공기-2
가능정반 : {'A': Timestamp('2024-02-01 00:00:00', freq='D'), 'B': Timestamp('2024-02-04 00:00:00', freq='D')}
최선조기착수 가능정반 : 2024-02-01 00:00:00 ---> ['A']
랜덤선택최선정반 : A - size_capa: 200, block_size:100, 잔여면적비율 :0.5
잔여면적비율 50.0%로 30% 이상이므로 정반 쪼개기 - 자식정반이름 :blue[**A_0**] / 자식정반면적 100.0 / 엄마정반면적-100.0
최종배정결과 - {'블록명': 'S1', '정반명': 'A', '착수일': '2024-02-01'}
(블록배치전) 수정블록리스트 : ['S1', 'S4', 'S3'], 수정정반리스트 : ['A', 

,블록명,정반명,착수일,표준공기,종료일,조립중량
0,S2,B,2024-02-01,3,2024-02-04,60
1,S1,A,2024-02-01,2,2024-02-03,50
2,S4,C,2024-02-01,2,2024-02-03,20
3,S3,A_0,2024-02-01,1,2024-02-02,30


In [229]:
result[1]

,A,B,C,A_0,A_0_1,C_2
2024-02-01,1,1,1,1,0,0
2024-02-02,1,1,0,1,0,0
2024-02-03,0,1,0,0,0,0
2024-02-04,0,0,0,0,0,0
2024-02-05,0,0,0,0,0,0
2024-02-06,0,0,0,0,0,0
2024-02-07,0,0,0,0,0,0
2024-02-08,0,0,0,0,0,0
2024-02-09,0,0,0,0,0,0
2024-02-10,0,0,0,0,0,0


In [221]:
result[2]

,A,B,C,A_0,C_1,A_0_2
2024-02-01,0.0,0.0,0.0,27,28,28
2024-02-02,0.0,0.0,0.0,26,27,27
2024-02-03,26.0,0.0,26.0,25,26,26
2024-02-04,25.0,25.0,25.0,24,25,25
2024-02-05,24.0,24.0,24.0,23,24,24
2024-02-06,23.0,23.0,23.0,22,23,23
2024-02-07,22.0,22.0,22.0,21,22,22
2024-02-08,21.0,21.0,21.0,20,21,21
2024-02-09,20.0,20.0,20.0,19,20,20
2024-02-10,19.0,19.0,19.0,18,19,19


In [222]:
result[3]

,A,B,C,A_0,C_1,A_0_2
2024-02-01,0,0,0,0,1,1
2024-02-02,0,0,0,1,0,0
2024-02-03,1,0,1,0,0,0
2024-02-04,0,1,0,0,0,0
2024-02-05,0,0,0,0,0,0
2024-02-06,0,0,0,0,0,0
2024-02-07,0,0,0,0,0,0
2024-02-08,0,0,0,0,0,0
2024-02-09,0,0,0,0,0,0
2024-02-10,0,0,0,0,0,0
